In [1]:
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

import pandas as pd

### Primeiramente, carrego a lista de municípios do tse direto do site oficial

In [2]:
_ = 'http://www.tse.jus.br/arquivos/tse-lista-de-municipios-do-cadastro-da-justica-eleitoral/at_download/file'

with urlopen(_) as _:
    _ = BytesIO(_.read())

with ZipFile(_) as _:
    with _.open('lista_municipios_justica_eleitoral.txt') as _:
        municipios_tse = pd.read_csv(_, encoding='latin1', sep=';')

municipios_tse.head()

,CÓDIGO,UF,NOME
0,1120,AC,ACRELÂNDIA
1,1570,AC,ASSIS BRASIL
2,1058,AC,BRASILÉIA
3,1007,AC,BUJARI
4,1015,AC,CAPIXABA


### E a lista de múnicipios do ibge, através do repositório git@github:kelvins/Municipios-Brasileiros

In [3]:
_ = 'https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/8730f8f9f90a47546f08b216484f5c336a1f531a/municipios_brasileiros.csv'

municipios_brasileiros = pd.read_csv(_)

municipios_brasileiros.head()

,codigo_ibge,nome_municipio,codigo_uf,uf,estado,capital,latitude,longitude
0,5200050,Abadia de Goiás,52,GO,Goiás,0,-16.75730,-49.4412
1,3100104,Abadia dos Dourados,31,MG,Minas Gerais,0,-18.48310,-47.3916
2,5200100,Abadiânia,52,GO,Goiás,0,-16.19700,-48.7057
3,3100203,Abaeté,31,MG,Minas Gerais,0,-19.15510,-45.4444
4,1500107,Abaetetuba,15,PA,Pará,0,-1.72183,-48.8788


### Note que ambas as listas contém o mesmo número de rows, desejávelmente seja uma relação de 1 pra 1 em cada lista

In [4]:
len(municipios_tse), len(municipios_brasileiros)

(5570, 5570)

### Indexo cada tabela por (uf, municipio), todos em uppercase

In [5]:
indexed_tse = (
    municipios_tse
    .set_index(
        municipios_tse
        [['UF', 'NOME']]
        .apply(tuple, axis='columns')
    )
    .sort_index()
)

indexed_tse.head()

,CÓDIGO,UF,NOME
"(AC, ACRELÂNDIA)",1120,AC,ACRELÂNDIA
"(AC, ASSIS BRASIL)",1570,AC,ASSIS BRASIL
"(AC, BRASILÉIA)",1058,AC,BRASILÉIA
"(AC, BUJARI)",1007,AC,BUJARI
"(AC, CAPIXABA)",1015,AC,CAPIXABA


In [6]:
indexed_ibge = (
    municipios_brasileiros
    .set_index(
        municipios_brasileiros
        [['uf', 'nome_municipio']]
        .applymap(str.upper)
        .apply(tuple, axis='columns')
    )
    .sort_index()
)

indexed_ibge.head()

,codigo_ibge,nome_municipio,codigo_uf,uf,estado,capital,latitude,longitude
"(AC, ACRELÂNDIA)",1200013,Acrelândia,12,AC,Acre,0,-9.82581,-66.8972
"(AC, ASSIS BRASIL)",1200054,Assis Brasil,12,AC,Acre,0,-10.92980,-69.5738
"(AC, BRASILÉIA)",1200104,Brasiléia,12,AC,Acre,0,-10.99500,-68.7497
"(AC, BUJARI)",1200138,Bujari,12,AC,Acre,0,-9.81528,-67.9550
"(AC, CAPIXABA)",1200179,Capixaba,12,AC,Acre,0,-10.56600,-67.6860


### Tento o join:

In [7]:
joined = indexed_tse.join(indexed_ibge, how='inner')
joined.head()

,CÓDIGO,UF,NOME,codigo_ibge,nome_municipio,codigo_uf,uf,estado,capital,latitude,longitude
"(AC, ACRELÂNDIA)",1120,AC,ACRELÂNDIA,1200013,Acrelândia,12,AC,Acre,0,-9.82581,-66.8972
"(AC, ASSIS BRASIL)",1570,AC,ASSIS BRASIL,1200054,Assis Brasil,12,AC,Acre,0,-10.92980,-69.5738
"(AC, BRASILÉIA)",1058,AC,BRASILÉIA,1200104,Brasiléia,12,AC,Acre,0,-10.99500,-68.7497
"(AC, BUJARI)",1007,AC,BUJARI,1200138,Bujari,12,AC,Acre,0,-9.81528,-67.9550
"(AC, CAPIXABA)",1015,AC,CAPIXABA,1200179,Capixaba,12,AC,Acre,0,-10.56600,-67.6860


### Ficam faltando:

In [8]:
len(indexed_tse) - len(joined)

52

### Analisando quais são:

In [9]:
indexed_tse2 = indexed_tse[~indexed_tse.index.isin(joined.index)]
indexed_tse2

,CÓDIGO,UF,NOME
"(AP, ÁGUA BRANCA DO AMAPARI)",6084,AP,ÁGUA BRANCA DO AMAPARI
"(BA, ARAÇÁS)",33383,BA,ARAÇÁS
"(BA, CAEM)",34010,BA,CAEM
"(BA, IUIU)",30660,BA,IUIU
"(BA, MUQUÉM DO SÃO FRANCISCO)",37460,BA,MUQUÉM DO SÃO FRANCISCO
"(BA, QUINJINGUE)",38199,BA,QUINJINGUE
"(BA, SANTA TEREZINHA)",38695,BA,SANTA TEREZINHA
"(BA, SANTO ESTEVÃO)",38750,BA,SANTO ESTEVÃO
"(CE, ITAPAGÉ)",14273,CE,ITAPAGÉ
"(ES, ATÍLIO VIVÁCQUA)",56138,ES,ATÍLIO VIVÁCQUA


In [10]:
indexed_ibge2 = indexed_ibge[~indexed_ibge.index.isin(joined.index)]
indexed_ibge2

,codigo_ibge,nome_municipio,codigo_uf,uf,estado,capital,latitude,longitude
"(AP, PEDRA BRANCA DO AMAPARI)",1600154,Pedra Branca do Amapari,16,AP,Amapá,0,0.777424,-51.9503
"(BA, ARAÇAS)",2902054,Araças,29,BA,Bahia,0,-12.220000,-38.2027
"(BA, CAÉM)",2905107,Caém,29,BA,Bahia,0,-11.067700,-40.4320
"(BA, IUIÚ)",2917334,Iuiú,29,BA,Bahia,0,-14.405400,-43.5595
"(BA, MUQUÉM DE SÃO FRANCISCO)",2922250,Muquém de São Francisco,29,BA,Bahia,0,-12.065000,-43.5497
"(BA, QUIJINGUE)",2925907,Quijingue,29,BA,Bahia,0,-10.750500,-39.2137
"(BA, SANTA TERESINHA)",2928505,Santa Teresinha,29,BA,Bahia,0,-12.769700,-39.5215
"(BA, SANTO ESTÊVÃO)",2928802,Santo Estêvão,29,BA,Bahia,0,-12.428000,-39.2505
"(CE, ITAPAJÉ)",2306306,Itapajé,23,CE,Ceará,0,-3.683140,-39.5855
"(ES, ATILIO VIVACQUA)",3200706,Atilio Vivacqua,32,ES,Espírito Santo,0,-20.913000,-41.1986


### Aplicando correções para o join:

In [11]:
def normalize(col):
    return (
        col
        .str.upper()
        .str.replace(r'[ÁÀÃ]', 'A')
        .str.replace(r'[ÉÊ]', 'E')
        .str.replace('Í', 'I')
        .str.replace('Ç', 'C')
        .str.replace(r'[ÓÔ]', 'O')
        .str.replace(r'[ÚÜ]', 'U')
        .str.replace('ELLO', 'ELO')
        .str.replace("D'O", "DO O")
        .str.replace("D'A", "DA A")
        .str.replace("T'A", "TA")
        .str.replace(' DE ', ' DO ')
        .str.replace('-', ' ')
        .str.replace(r'.+\((.+)\).*', r'\1')
    )

In [12]:
indexed_tse2_fixed = indexed_tse2.set_index(indexed_tse2[['UF', 'NOME']].assign(
    NOME=normalize(indexed_tse2.NOME)
).apply(tuple, axis='columns'))

indexed_tse2_fixed.head()

,CÓDIGO,UF,NOME
"(AP, AGUA BRANCA DO AMAPARI)",6084,AP,ÁGUA BRANCA DO AMAPARI
"(BA, ARACAS)",33383,BA,ARAÇÁS
"(BA, CAEM)",34010,BA,CAEM
"(BA, IUIU)",30660,BA,IUIU
"(BA, MUQUEM DO SAO FRANCISCO)",37460,BA,MUQUÉM DO SÃO FRANCISCO


In [13]:
indexed_ibge2_fixed = indexed_ibge2.set_index(indexed_ibge2[['uf', 'nome_municipio']].assign(
    nome_municipio=normalize(indexed_ibge2.nome_municipio)
).apply(tuple, axis='columns'))

indexed_ibge2_fixed.head()

,codigo_ibge,nome_municipio,codigo_uf,uf,estado,capital,latitude,longitude
"(AP, PEDRA BRANCA DO AMAPARI)",1600154,Pedra Branca do Amapari,16,AP,Amapá,0,0.777424,-51.9503
"(BA, ARACAS)",2902054,Araças,29,BA,Bahia,0,-12.220000,-38.2027
"(BA, CAEM)",2905107,Caém,29,BA,Bahia,0,-11.067700,-40.4320
"(BA, IUIU)",2917334,Iuiú,29,BA,Bahia,0,-14.405400,-43.5595
"(BA, MUQUEM DO SAO FRANCISCO)",2922250,Muquém de São Francisco,29,BA,Bahia,0,-12.065000,-43.5497


In [14]:
joined2 = indexed_tse2_fixed.join(indexed_ibge2_fixed, how='inner')
joined2.head()

,CÓDIGO,UF,NOME,codigo_ibge,nome_municipio,codigo_uf,uf,estado,capital,latitude,longitude
"(BA, ARACAS)",33383,BA,ARAÇÁS,2902054,Araças,29,BA,Bahia,0,-12.2200,-38.2027
"(BA, CAEM)",34010,BA,CAEM,2905107,Caém,29,BA,Bahia,0,-11.0677,-40.4320
"(BA, IUIU)",30660,BA,IUIU,2917334,Iuiú,29,BA,Bahia,0,-14.4054,-43.5595
"(BA, MUQUEM DO SAO FRANCISCO)",37460,BA,MUQUÉM DO SÃO FRANCISCO,2922250,Muquém de São Francisco,29,BA,Bahia,0,-12.0650,-43.5497
"(BA, SANTO ESTEVAO)",38750,BA,SANTO ESTEVÃO,2928802,Santo Estêvão,29,BA,Bahia,0,-12.4280,-39.2505


### Ainda faltam:

In [15]:
len(indexed_tse2)- len(joined2)

18

### Analisando:

In [16]:
indexed_tse3 = indexed_tse2_fixed[~indexed_tse2_fixed.index.isin(joined2.index)]
indexed_tse3

,CÓDIGO,UF,NOME
"(AP, AGUA BRANCA DO AMAPARI)",6084,AP,ÁGUA BRANCA DO AMAPARI
"(BA, QUINJINGUE)",38199,BA,QUINJINGUE
"(BA, SANTA TEREZINHA)",38695,BA,SANTA TEREZINHA
"(CE, ITAPAGE)",14273,CE,ITAPAGÉ
"(PA, ELDORADO DOS CARAJAS)",4120,PA,ELDORADO DOS CARAJÁS
"(PA, SANTA ISABEL DO PARA)",5290,PA,SANTA ISABEL DO PARÁ
"(PB, SANTAREM)",19666,PB,SANTARÉM
"(PB, SAO DOMINGOS DO POMBAL)",19429,PB,SÃO DOMINGOS DE POMBAL
"(PE, IGUARACI)",24376,PE,IGUARACI
"(PE, SAO CAITANO)",25615,PE,SÃO CAITANO


In [17]:
indexed_ibge3 = indexed_ibge2_fixed[~indexed_ibge2_fixed.index.isin(joined2.index)]
indexed_ibge3

,codigo_ibge,nome_municipio,codigo_uf,uf,estado,capital,latitude,longitude
"(AP, PEDRA BRANCA DO AMAPARI)",1600154,Pedra Branca do Amapari,16,AP,Amapá,0,0.777424,-51.9503
"(BA, QUIJINGUE)",2925907,Quijingue,29,BA,Bahia,0,-10.750500,-39.2137
"(BA, SANTA TERESINHA)",2928505,Santa Teresinha,29,BA,Bahia,0,-12.769700,-39.5215
"(CE, ITAPAJE)",2306306,Itapajé,23,CE,Ceará,0,-3.683140,-39.5855
"(PA, ELDORADO DO CARAJAS)",1502954,Eldorado do Carajás,15,PA,Pará,0,-6.103890,-49.3553
"(PA, SANTA IZABEL DO PARA)",1506500,Santa Izabel do Pará,15,PA,Pará,0,-1.296860,-48.1606
"(PB, JOCA CLAUDINO)",2513653,Joca Claudino,25,PB,Paraíba,0,-6.483620,-38.4764
"(PB, SAO DOMINGOS)",2513968,São Domingos,25,PB,Paraíba,0,-6.803130,-37.9488
"(PE, IGUARACY)",2606903,Iguaracy,26,PE,Pernambuco,0,-7.832220,-37.5082
"(PE, SAO CAETANO)",2613107,São Caetano,26,PE,Pernambuco,0,-8.337630,-36.2869


### Esses últimos podem ser joinados por ordem alfabética:

In [18]:
joined3 = indexed_tse3.reset_index(drop=True).join(indexed_ibge3.reset_index(drop=True))
joined3

,CÓDIGO,UF,NOME,codigo_ibge,nome_municipio,codigo_uf,uf,estado,capital,latitude,longitude
0,6084,AP,ÁGUA BRANCA DO AMAPARI,1600154,Pedra Branca do Amapari,16,AP,Amapá,0,0.777424,-51.9503
1,38199,BA,QUINJINGUE,2925907,Quijingue,29,BA,Bahia,0,-10.750500,-39.2137
2,38695,BA,SANTA TEREZINHA,2928505,Santa Teresinha,29,BA,Bahia,0,-12.769700,-39.5215
3,14273,CE,ITAPAGÉ,2306306,Itapajé,23,CE,Ceará,0,-3.683140,-39.5855
4,4120,PA,ELDORADO DOS CARAJÁS,1502954,Eldorado do Carajás,15,PA,Pará,0,-6.103890,-49.3553
5,5290,PA,SANTA ISABEL DO PARÁ,1506500,Santa Izabel do Pará,15,PA,Pará,0,-1.296860,-48.1606
6,19666,PB,SANTARÉM,2513653,Joca Claudino,25,PB,Paraíba,0,-6.483620,-38.4764
7,19429,PB,SÃO DOMINGOS DE POMBAL,2513968,São Domingos,25,PB,Paraíba,0,-6.803130,-37.9488
8,24376,PE,IGUARACI,2606903,Iguaracy,26,PE,Pernambuco,0,-7.832220,-37.5082
9,25615,PE,SÃO CAITANO,2613107,São Caetano,26,PE,Pernambuco,0,-8.337630,-36.2869


### O único estranho é Santarém -> Joca Claudino em PB, mas googlando confirmo que o município mudou de nome

In [19]:
final = pd.concat([joined, joined2, joined3]).sort_values(['UF', 'NOME']).reset_index(drop=True)[[
    'CÓDIGO',
    'UF',
    'NOME',
    'capital',
    'codigo_ibge',
]].rename(columns={
    'CÓDIGO': 'codigo_tse',
    'UF': 'uf',
    'NOME': 'nome_municipio',
})

final.head()

,codigo_tse,uf,nome_municipio,capital,codigo_ibge
0,1120,AC,ACRELÂNDIA,0,1200013
1,1570,AC,ASSIS BRASIL,0,1200054
2,1058,AC,BRASILÉIA,0,1200104
3,1007,AC,BUJARI,0,1200138
4,1015,AC,CAPIXABA,0,1200179


In [20]:
final.to_csv('municipios_brasileiros_tse.csv', index=None)

with open('municipios_brasileiros_tse.json', 'w') as fp:
    fp.write(
        '[\n' +
        ','.join(final.to_json(orient='records', force_ascii=False, lines=True).split('\n')) +
        '\n]'
    )